# Overview

This notebook walks through how PyDough can be leveraged to perform "WHAT-IF" analysis. At its core, we are typically looking for insights on questions that are much too complex or abstract, such as "How can we improve sales in Q2?" As a result, this approach incentivizes faster iteration to rapidly explore various scenarios in the hope of uncovering insights that can resolve our abstract problem. 

We believe that PyDough is ideal for these types of questions because PyDough can be used to solve common intermediate problems and quickly iterate betwen alterantive versions. Rather than focusing on a single query to answer a question at hand, building components allows more proportional scaling and more easily modifying the scenario. The PyDough team is optimistic that such an approach can make humans more productive and LLMs more accurate.

In this notebook we will focus on addressing a single question, but we will illustrate the principles that make such a question convenient for rapid iteration. Towards the end of this notebook, we will then modify this base question in the spirit of "WHAT-IF", but this will not be a complete what if analysis. We encourage you to explore new ways to extend this investigation by consulting the [PyDough documentation](../../documentation/usage.md).

In [ ]:
%load_ext pydough.jupyter_extensions

In [ ]:
import pydough
import pandas as pd

In [ ]:
# Setup demo metadata
pydough.active_session.load_metadata_graph("../metadata/tpch_demo_graph.json", "TPCH");
pydough.active_session.connect_database("sqlite", database="../../tpch.db");
# Avoid scientific notation
pd.options.display.float_format = '{:.6f}'.format

## Schema Setup

For this demo we will be working in TPC-H benchmark schema. The actual data for this benchmark is generated in SQLite using the standard TPC-H data generation tools. The underlying schema of this data matches this example image from [TPC Benchmark H Standard Specification](https://www.tpc.org/tpc_documents_current_versions/pdf/tpc-h_v2.17.1.pdf). For this example we will only be using the **LINEITEM** and **ORDERS** tables.

![TPC-H schema from the Specification Document as of December 12, 2024](../images/tpc_h_schema.png)

## Problem Statement

Let's say that we are focused on identifying **ways to increase revenue in the next calendar year**. One idea that we might have is to explore items that are a very small percentage of a total order price as changes to these items may not have a meaningful impact on the customer but could result in larger revenue aggregate.   

As a first scenario, we are going to **identify the 5 lines in our entire order sheet that represent the smallest percentage of their corresponding order's revenue**. 

Here is a possible SQL query that could we be used to answer this question and in constrast here is the corresponding PyDough.

### SQL

```SQL
Select
    (L_EXTENDED_PRICE * (1 - L_DISCOUNT)) / order_total as revenue_ratio,
    orderkey,
    l_linenumber as line_number
from lines
inner join (
    select
        o_orderkey as order_key,
        SUM(L_EXTENDED_PRICE * (1 - L_DISCOUNT)) as order_total
    from orders
    inner join lines
    on l_orderkey = o_orderkey
    group by o_orderkey
)
on l_orderkey = order_key
order by revenue_ratio ASC, order_key DESC, line_number DESC
LIMIT 5
```

### PyDough

```python
revenue_def = extended_price*(1-discount)
orders.CALCULATE(total_line_price=SUM(lines.CALCULATE(line_price=revenue_def).line_price)).lines.CALCULATE(
    revenue_ratio=revenue_def / total_line_price, 
    order_key=order_key, 
    line_number=line_number
).TOP_K(5, by=(revenue_ratio.ASC(), order_key.DESC(), line_number.DESC()))
```

The example SQL requires a nested subquery to answer, which can easily become complicated and unmanagable as questions scale in size. It also requires explicitly representing the join information where a decision like **LEFT** vs **INNER** join impacts correctness and is a function of the data.

In constrast, PyDough has a simpler representation, avoiding some of the redundancy in the SQL code and the join information is encoded entirely in the metadata.

## PyDough Solution

While we can just execute the PyDough example above, we are going to take a different approach to show how one might generate this query in a "WHAT-IF" manner. This approach will be a longer solution, but will be helpful to enable faster iteration once we modify the question.

We are opting to demonstrate this because:
1. We believe this is a good representation for how PyDough can be leveraged to gradually handle increasing question complexity.
2. We believe this reflects an investigative approach to query generation, where someone may understand at a high-level what needs to be done, but not necessary that "path" to get there.

To do this we will first need to define revenue. Here we will say that the revenue is the price information after removing any discounts.

In [ ]:
%%pydough

revenue_def = extended_price*(1-discount)

This might seem shocking. We have defined `revenue_def` out of nowhere using an `extended_price` and `discount`. What has actually happened here is that we have generated what is called a `Contextless Expression`. This fundamental building block is the key to PyDough's composability.

On its own this expression doesn't mean anything. In fact if we inspect this object in regular PyDough we will see that PyDough itself has a lot of questions.

In [ ]:
revenue_def

As you see, PyDough now knows that this expression is composed of `extended_price` and `discount`, but it doesn't know **WHICH** `extended_price` and `discount`. To ultimately develop a legal PyDough statement, we will need to bind uses of this expression to a context that can access `extended_price` and `discount`.

This might seem very minor, but this allows us to define definitions up front, allowing reuse in vastly different contexts.

Now let's use this definition to compute the total revenue.

In [ ]:
%%pydough

total_revenue = SUM(lines.CALCULATE(line_revenue=revenue_def).line_revenue)
total_revenue

Now this expression is more meaningful, but it actually still doesn't have a context. If we assign this statement to the global context, our actual TPCH graph, then we can compute the total revenue across all lines.

In [ ]:
%%pydough

pydough.to_df(TPCH.CALCULATE(total_line_revenue=total_revenue))

In practice though, this may not solve our core question. Instead, we may want to apply a different **context**, say for example total_revenue for each order. We can instead represent that as follows.

In [ ]:
%%pydough

order_total_price = orders.CALCULATE(order_revenue=total_revenue)
pydough.to_df(order_total_price)

Notice that are able to reuse the exact same code, but by swapping the context we can ultimately modify the semantics. This makes testing the underlying behavior much more scalable. To ask is this statement correct, we can instead compose our question to ask:
* Is this underlying expression correct?
* Is this context correct?

Since these can be verify independently, we can develop greater confidence in our question since it arises from composable building blocks. We could also generate selected contexts to build clear testing. Here we show reusing the same code but with a selection of 5 lines. If we instead provide a testing context it could be done without any code rewrite

In [ ]:
%%pydough
# Compute the sum of the first 5 line numbers, which can be known for testing.
top_five_lines = lines.TOP_K(5, by=(line_number.ASC(), order_key.ASC()))
top_five_line_price = TPCH.CALCULATE(total_line_revenue=SUM(top_five_lines.CALCULATE(line_revenue=revenue_def).line_revenue))
pydough.to_df(top_five_line_price)

Now let's return to extending our question. Building able to compute order sums is great, but we care about results per line. As a result, now we can even extend our orders to an additional context within lines. We will once again define more defintions. Our ratio definition will now ask us to propagate our previous `order_revenue` that we computed (down-streamed from an ancestor context) and compare it to the result of `revenue_def`.

In [ ]:
%%pydough

ratio = revenue_def / order_revenue

Now we will build our final question by moving from the context of the orders to the context of orders and lines together. Since Orders -> Lines is a One to Many relationship, this places us in the context of lines with some additional information.

For actually fully solving our prior question, we will compute the ratio and then select 5 smallest ratio value, breaking ties with a combination of the order number and line number.

In [ ]:
%%pydough

line_ratios = order_total_price.lines.CALCULATE(revenue_ratio=ratio, order_key=order_key, line_number=line_number)
lowest_ratios = line_ratios.TOP_K(5, by=(revenue_ratio.ASC(), order_key.DESC(), line_number.DESC()))

In [ ]:
pydough.to_df(lowest_ratios)

Now we have resolved to solution to our underlying question. We can save this result to a Python variable, as the output is already a Pandas DataFrame.

## Additional Exploration

A natural followup question is to try understand how we can leverage our previous work into followup questions. One option would be to do a deeper dive into the parts that represent these orders, but first let's consider the opposite, where we want to instead look at the items that are the largest percentage of orders. Perhaps rather than raising the price on existing products we should be determining how to design new additions to products.

In [ ]:
%%pydough

highest_ratios = line_ratios.TOP_K(5, by=(revenue_ratio.DESC(), order_key.DESC(), line_number.DESC()))
pydough.to_df(highest_ratios)

While we could quickly reach our solution, one problem that we encounter is that the items with the highest ratio are all single item purchases. This is one clear direction we can explore, but it may also include a lot of small cost items that are purchased individually.

Here let's expand our scenario by asking **what if we only consider the lines that are part of orders with more than 3 lines**? We can do this by just extending many of our existing definitions and adding a filter on the number of lines

In [ ]:
%%pydough

total_lines = COUNT(lines)
order_total_price = orders.CALCULATE(order_revenue=total_revenue, line_count=total_lines)
line_ratios = order_total_price.lines.CALCULATE(
    revenue_ratio=ratio, 
    line_count=line_count, 
    order_key=order_key, 
    line_number=line_number
)
filtered_ratios = line_ratios.WHERE(line_count > 3).CALCULATE(revenue_ratio, order_key, line_number)

Now that we have our filtered results, we can once again compute our ratios. This should allow us to test our hypothesis that we can should explore additional parts for these "big-ticket" items.

In [ ]:
%%pydough

highest_ratios = filtered_ratios.TOP_K(
    5, by=(revenue_ratio.DESC(), order_key.DESC(), line_number.DESC())
)
pydough.to_df(highest_ratios)

We have now successfully started extending our exploration and are free to tackle any number of followup questions, such as diving into the parts that fulfill these orders or considering different order sizes. Contrast this with the approach we would need to take with a SQL query would requires either modifying our past work or copying the results and repeating the logic, which may evolve over time.

We encourage you to explore extending this notebook to examine additional ways in which PyDough can facilitate rapid "WHAT-IF" analysis.